In [25]:
import pandas as pd

#üzerinde önişleme yapacağım verisetime eriştim.
df = pd.read_excel("DATASET.xlsx")


In [26]:
#"Script" ve "Until" olan activityType değerlerine sahip satırları kaldırdım.
df = df[~df['activityType'].isin(['Script', 'Until'])]

#status değeri "Inactive" olan satırları veri setinden çıkardım.
df = df[df['status'] != 'Inactive']

#verisetinde eğer 13 sütundan fazlası boşsa bir satırda o satırları kaldırdım.
df = df[df.isnull().sum(axis=1) <= 13]

#veri temizliği işlemlerini gerçekleştirdim.

In [27]:
#Self-hosted IR şirket içi sunucularda çalıştığı için integrationRuntime_location sütunundaki değerlerine “on-prem” yazdım.
df.loc[df['integrationRuntime_type'] == 'SelfHosted', 'integrationRuntime_location'] = 'On-prem'

#Boş olan interimRowsCopied sütunundaki boş değerleri, aynı satırdaki rowsCopied değerleriyle doldurdum.
condition = df['rowsCopied'].notna() & df['interimRowsCopied'].isna()
df.loc[condition, 'interimRowsCopied'] = df.loc[condition, 'rowsCopied']

#Boş olan interimDataWritten sütunundaki boş değerleri, aynı satırdaki dataWritten değerleriyle doldurdum.
condition2 = df['dataWritten'].notna() & df['interimDataWritten'].isna()
df.loc[condition2, 'interimDataWritten'] = df.loc[condition2, 'dataWritten']


In [28]:
#durationInQueue boş olan satırlarda, queuingDuration - 1 değerini yazdırdım.
#queuingDuration 0  ise durationInQueue da 0 olması şartını ekledim (-1 olmamalı).
mask_null = df['durationInQueue'].isnull()
df.loc[mask_null & (df['queuingDuration'] == 0), 'durationInQueue'] = 0
df.loc[mask_null & (df['queuingDuration'] != 0), 'durationInQueue'] = df['queuingDuration'] - 1

#timeToFirstByte sütununda boş olan değerleri bu sütunun en çok tekrar eden değeri ile doldurdum.
most_frequent_value = df['timeToFirstByte'].mode()[0]
df['timeToFirstByte'] = df['timeToFirstByte'].fillna(most_frequent_value)

#rowsRead 0 olanların eğer interimRowsCopied ve rowsCopied sütunlarındaki değerler boş ise 0 ile doldurdum.
mask = df['rowsRead'] == 0
df.loc[mask & df['interimRowsCopied'].isnull(), 'interimRowsCopied'] = 0
df.loc[mask & df['rowsCopied'].isnull(), 'rowsCopied'] = 0

#sinkPeakConnections sütununda boş olan değerleri bu sütunun modu ile doldurdum.
most_frequent_value = df['sinkPeakConnections'].mode()[0]
df['sinkPeakConnections'] = df['sinkPeakConnections'].fillna(most_frequent_value)


In [29]:
#önişleme sonrası düzenlenmiş verisetimi elde ediyorum.
df.to_excel("DATASET_LASTFORM.xlsx", index=False)
